# Friedmann Equations
We've already seen the equations in class.  Let's develop some intuition about how the parameters influence the time evolution of the scale factor $a$.  We begin with the differential equation

$\frac{\ddot{a}}{a} = - \frac{4 \pi G}{3} \left( \rho + \frac{3P}{c^2} \right) + \frac{\Lambda}{3}$


Where $G$ is Newton's Gravitational constant, $P$ is the pressure, $\rho$ is the density, $c$ is the speed of light and $\Lambda$ is the Cosmological Constant.

In [ ]:
#this cell loads lots of useful functions for us
from pylab import *

#fundamental units
s=1.0 #time unit is seconds
cm=1.0 #distance unit is cm
erg=1.0 #energy unit is erg
g=1.0 #mass unit gram

#constants
sec_yr = 365.25*24*3600 # seconds per year
Mpc=3.086e24*cm #cm
G=6.6743e-8*cm*cm*cm/g/s/s #cgs units
km=1e5*cm
H0_h=100.0*km/s/Mpc #1/s

Below is a function allows us to simulate the time evolution of the Friedmann equations for different values of the parameters that govern the model. The way it works is simple, and the numerical integration it performs is the simplest method possible -- which means that we won't be able to follow the integration when the scale factor _a_ approaches zero.

It follows the evolution in two parts and then joins them together at the end.  Always starting from "today" and evolving back in time until _a < 0.01_ and then forward in time for a fixed interval of time, or unitl _a < 0.01_ if that is possible given the parameters.

In [ ]:
def cosmology(omega_m=1.0, omega_r=1.0, omega_L=1.0, w = -1.0, h=0.7, tmax = 10.):
    ''' integrate forwards and backwards the Friedmann equation
        to obtain the evolution of the scale factor, its time derivative,
        redshift z, and the deceleration parameter q'''

    #some computed constants, values depend on the inputs
    H0 = H0_h * h
    rho_crit = (3.*H0*H0)/(8.*pi*G)
    matter = omega_m * rho_crit
    radiation = omega_r * rho_crit
    darkEnergy = omega_L * rho_crit
    darkEnergyFactor = 3.0 * w + 1.0
    darkEnergyPower = 3.0 * w + 2.0

    dt = 1.0e6*sec_yr # 1 Myr timestep

    # Run simulation backward from present day until scale factor is < 0.01
    lResultsBackward = []

    t = 0.0
    a = 1.0
    adot = H0

    q0 = 0.5*omega_m - omega_L

    while a > 0.01:
        matterTerm = matter * pow(a,-2)
        radiationTerm = 2.0 * radiation * pow(a,-3)
        darkEnergyTerm = darkEnergyFactor * darkEnergy * pow(a,-darkEnergyPower)

        a2dot = -(4.0/3.0) * pi * G * (matterTerm + radiationTerm + darkEnergyTerm)
        adot = adot - dt * a2dot
        a = a - dt * adot
        t = t - dt
        z = 1./a - 1.
        q = -a * a2dot / (adot*adot)

        lResultsBackward.append((t/sec_yr/1e9, a, z, q))

    # Run simulation forward from present day for another 50 billion years
    # or until the scale factor falls below 0.01
    lResultsForward = []

    t = 0.0
    a = 1.0
    adot = H0_h * h

    while t < tmax*1e9*sec_yr and a > 0.01:
        matterTerm = matter * pow(a,-2)
        radiationTerm = 2.0 * radiation * pow(a,-3)
        darkEnergyTerm = darkEnergyFactor * darkEnergy * pow(a,-darkEnergyPower)

        a2dot = -(4.0/3.0) * pi * G * (matterTerm + radiationTerm + darkEnergyTerm)
        adot = adot + dt * a2dot
        a = a + dt * adot
        t = t + dt
        z = 1./a - 1.
        q = -a * a2dot / (adot*adot)

        lResultsForward.append((t/sec_yr/1.0e9, a, z, q))

    # Write results to an array with names columns for ease of use

    lResultsBackward.reverse()
    lResults = lResultsBackward + [(0.0, 1.0, 0., q0)] + lResultsForward
    return array(lResults, dtype=[('t', float64), ('a', float64), ('z', float64), ('q', float64) ])


We can use this function to make a model for our favored model.  According to the Planck mission, the current favored values are as follows:

*   $H_0 = 67.74$ km/s/Mpc meaning $h = 0.6774$
*   $\Omega_m = 0.3089$
*   $\Omega_{\Lambda} = 0.6911$
*   $\Omega_r \approx 0$
*   $w = -1$

The above function takes each of these values as arguments:

*   h
*   omega_m
*   omega_r
*   omega_L
*   w

It returns (gives us back) an array containing the time coordinate (units: Gyr), the scale factor $a$, the redshift $z$, and the 'deceleration parameter' $q$.

Let's say we run the function and assign its output to x:

```
x = cosmology()
```

then we can access the results by name, e.g.,

```
a = x['a']
t = x['t']
z = x['z']
q = x['q']
```

Here we run the model for the favored values and then make a plot of a vs. t.


In [ ]:
x = cosmology(omega_m = 0.3089, omega_r = 0., omega_L = 0.6911, w = -1., h=0.6774)

In [ ]:
figure(1,figsize=(10,10))
plot(x['t'], x['a'],label="Lambda-CDM")
grid()
xlabel('t [Gyr]',fontsize=20)
xticks(fontsize=20)
yticks(fontsize=20)
legend()
ylabel(r'scale factor $a$',fontsize=20);

Notice that $t=0$ is "today".  What is the earliest point in the evolution?

In [ ]:
print('The earliest point in the simulation occured {0:6.3f} Gyr ago when the scale factor was {1:6.4f}.'.format(abs(x['t'][0]), x['a'][0]))

In [ ]:
print("At this time the redshift was z = {0:10.3f}.".format(x['z'][0]))

Let's now make a plot of the redshift as a function of time.  We'll plot the redshift on a log-scale so that we can see better how much it changes in the early universe compared to today.

In [ ]:
figure(2,figsize=(10,10))
plot(x['t'], x['z'])
yscale('log')
xlabel('look back time [Gyr]',fontsize=20)
xlim(-14,0)
grid()
axhline(0,ls=':')
xticks(fontsize=20)
yticks(fontsize=20)
ylabel(r'redshift $z$',fontsize=20);

# Exploring Different Models

Now let's look at how changing the various parameters changes the evolution.  First, a simple one where we change the value of $h$ and leave everything else the same.

In [ ]:
a1 = cosmology(omega_m = 0.3089, omega_r = 0., omega_L = 0.6911, w = -1., h=0.5)
a2 = cosmology(omega_m = 0.3089, omega_r = 0., omega_L = 0.6911, w = -1., h=0.75)
a3 = cosmology(omega_m = 0.3089, omega_r = 0., omega_L = 0.6911, w = -1., h=1.0)

figure(3,figsize=(10,10))
plot(a1['t'], a1['a'],label="h=0.50")
plot(a2['t'], a2['a'],label="h=0.75")
plot(a3['t'], a3['a'],label="h=1.00")
xlabel('t [Gyr]',fontsize=20)
xticks(fontsize=20)
yticks(fontsize=20)
legend(fontsize=20)
ylabel(r'scale factor $a$',fontsize=20);

__How does changing $h$ influence the age of the Universe?__



## Einstein-de Sitter model

The EdS model has only one component, mass.  Let's run a suite of models where we vary $\Omega_m$ and see how it influences the evolution of the Universe.  

In [ ]:
figure(4,figsize=(10,10))

masses=linspace(0,20,21)
for m in masses:
  x=cosmology(omega_m = m, omega_r = 0, omega_L = 0, tmax=20)
  plot(x['t'], x['a'])

xlabel('t [Gyr]',fontsize=20)
xticks(fontsize=20)
yticks(fontsize=20)
title(r"Varying $\Omega_m$",fontsize=20)
ylabel(r'scale factor $a$',fontsize=20);

__How does changing $\Omega_m$  influence the age of the universe?__

## Vary Radiation

In [ ]:
figure(4,figsize=(10,10))

rads=linspace(0.01,0.5,10)
for r in rads:
  x=cosmology(omega_m = 0.3, omega_r = r, omega_L = 0.7-r)
  plot(x['t'], x['a'])

xlabel('t [Gyr]',fontsize=20)
xticks(fontsize=20)
yticks(fontsize=20)
title(r"Varying $\Omega_r$",fontsize=20)
ylabel(r'scale factor $a$',fontsize=20);

## Vary the Cosmological Constant

In [ ]:
figure(5,figsize=(10,10))

Lambdas=linspace(0.1,1.5,10)
for L in Lambdas:
  x=cosmology(omega_m = 0.3, omega_r = 0, omega_L = L)
  plot(x['t'], x['a'])

xlabel('t [Gyr]',fontsize=20)
xticks(fontsize=20)
yticks(fontsize=20)
title(r"Varying $\Omega_{\Lambda}$",fontsize=20)
ylabel(r'scale factor $a$',fontsize=20);

__How does varing the cosmological constant affect the age of the universe?__

## Explore!

Try changing other things.  Come up with a set of parameters of your own choosing and describe the evolution of the Universe under those conditions.